# Crawling - KCIA (Result File: ing_name.csv)
대한 화장품 협회 웹사이트에서 제공하는 '성분사전'에서 성분정보 수집하기<br>
https://kcia.or.kr/cid/main/

### [ 목적 ]
- 최종적으로 제품 및 주성분 목록을 수집 시 활용할 사이트: InciDecoder
- InciDecoder 웹사이트에서 성분 영문명을 사용하여 특정 성분 페이지에 직접 접근 가능
- InciDecoder 성분 페이지 접근에 활용할 화장품 성분의 영문명을 KCIA 웹사이트에서 수집 


### [ KCIA 웹사이트 구성 및 특징 ]
1. 화장품 성분 목록이 1~2038개 페이지로 구성
   
2. 페이지 접근 url 에서 페이지 숫자{num}로 특정 페이지에 직접 접근 가능<br>
   https://kcia.or.kr/cid/search/ingd_list.php?page={num}
   
3. 각 페이지에서 하기 항목들을 html table 형식으로 제공<br>

   
   |성분코드|성분명|영문명|CAS No|구명칭|
   |---|---|---|---|---|

### [ 데이터 수집 방법 - <span style='color:green'>Tool</span> ]
1. 수집할 항목들을 컬럼명으로 지정하여 빈 데이터 프레임 생성데이터 프레임 명:'ing_df' - <span style='color: green'>pandas</span>
   
2. 1~2038 페이지에 url로 접근 - <span style='color: green'>requests</span>
   
3. 각 페이지 html 불러오기 - <span style='color: green'>BeautifulSoup</span>
   
4. html 소스에 접근하여 class가 bbs_list인 테이블을 받아오기 - <span style='color: green'>BeautifulSoup</span>,<span style='color: green'>parser_functions.make2d</span>
   
5. 각 테이블에서 수집한 정보를 임시 데이터 프레임으로 담은 후, 사전에 생성한 'ing_df' 데이터에 누적 (concat)

### [ 수집한 raw data 형태 ]

||성분코드|성분명|영문명|CAS No|구명칭|
|---|---|---|---|---|---|
|1|5|류신|Leucine|328-39-2(DL-)\r\n61-90-5(L-)|---|
|2|4|루핀아미노산|Lupine Amino Acids|---|---|
|3|3|구멍쇠미역추출물|Agarum Cribrosum Extract|---|---|
|4|2|가지열매추출물|Solanum Melongena (Eggplant) Fruit Extract|84012-19-1|가지추출물|---|---|
|5|1|가공소금|---|---|---|
|...|...|...|...|...|....|

### [ 데이터 확인 및 전처리 내용 ]
- 성분코드의 컬럼의 마지막 행 번호와 데이터 프레임의 행 개수 상이<br>(성분코드 마지막 행 번호:22996 vs 데이터 프레임의 행 개수: 20376)<br>
  
  <span style='color:red'>&rarr;</span> set 자료구조의 차집합을 통하여 빠진 성분코드 번호가 있는지 확인<br>
   <span style='color:red'>&rarr;</span> 수집한 데이터의 성분코드 번호에 누락된 번호가 있음<br>

- 수집 목적에 따라 영문명을 제공하지 않는 경우 사용 결측치로 판단
   |컬럼|계|
   |---|---|
   |성분명|20,376|
   |영문명|19,414|<br>

- 수집 목적에 따라 KCIA에 기재된 성분 영문명을 InciDecoder에서 제공하는 형식으로 전처리 필요:
  - 식물의 학명 뒤에 따라오는 괄호안 일반 명칭 삭제<ve>
  - 전체 소문자 처리
  - 공백 및 "/" 문자를 "-"로 변경
  
### [ 최종 파일 및 특징 ] 
파일명: ing_name.csv

|성분코드|성분명|영문명|formatted_영문명|
|---|---|---|---|
|1|가공소금|NaN|NaN|
|2|가재열매추출물|Solanum Melongena (Eggplant) Fruit Extract|solanum-melongena -fruit-extract|
|...|...|...|...|
|6|류코노스톡/무발효여과물|Leuconostoc/Radish Root Ferment Filtrate|leuconostoc-radish-root-ferment-filtrate|

- KCIA 웹사이트에서 수집한 성분 데이터 수: 20376개
- 결측치 (영문명을 제공하지 않는 성분) 제거를 통해 최종적으로 수집한 성분 데이터 수: 19414개

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
import requests

In [4]:
# kcia의 성분사전 테이블 형태로 빈 데이터 프레임을 미리 하나 만든다
cols = ['성분코드', '성분명', '영문명', 'CAS No', '구명칭']
ing_df = pd.DataFrame(columns=cols)
ing_df

,성분코드,성분명,영문명,CAS No,구명칭


In [5]:
for num in tqdm_notebook(range(1, 2039)): # 1~2038의 페이지로 구성되어있다
    url = f"https://kcia.or.kr/cid/search/ingd_list.php?page={num}" # url이 뒤의 page num만 바뀜
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

    data = soup.find('table',{"class" : "bbs_list"}) # class가 bbs_list인 테이블을 찾아와서
    data = parser_functions.make2d(data)[1:] # parser_functions의 make2d로 받아온다

    tmpdf = pd.DataFrame(data, columns=cols) # 임시 데이터 프레임을 만들어 각 페이지 별 테이블 정보를 담아서
    ing_df = pd.concat([ing_df ,tmpdf]) # 위에 만들어 놓은 빈 데이터프레임이랑 concat 하는 식으로 누적시킨다

ing_df

  0%|          | 0/2038 [00:00<?, ?it/s]

,성분코드,성분명,영문명,CAS No,구명칭
0,23249,흑효모/매스틱검추출물발효여과물,,,
1,23248,삼뿌리수,,,
2,23247,약모밀잎소포,,,
3,23243,효모/헥사펩타이드-11발효여과추출물,Saccharomyces/Hexapeptide-11 Ferment Filtrate ...,161258-30-6,
4,23242,비타민나무열매수,Hippophae Rhamnoides Fruit Water,,
...,...,...,...,...,...
5,10,리날릴아세테이트,Linalyl Acetate,115-95-7,
6,9,리날룰,Linalool,78-70-6,
7,8,리나칸투스 콤무니스추출물,Rhinacanthus Communis Extract,,
8,7,백혈구추출물,Leukocyte Extract,,류코사이트추출물


In [15]:
# 성분코드는 22996이 마지막인데, 컬럼 개수는 20376개다
# set 자료구조의 차집합을 통해 빠져있는 성분코드가 있는지 확인
set(list(range(1,22997))) - set(list(ing_df['성분코드'].astype('int'))); # 비교를 위해 str인 성분코드를 int로 바꿈
# 해당 번호들을 사이트에서 직접 확인해보니 빠져있는게 맞았다

In [7]:
ing_df = ing_df.astype({'성분코드':'int'})
ing_df = ing_df[['성분코드', '성분명', '영문명']].set_index('성분코드').sort_index()
ing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20380 entries, 6 to 23249
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   성분명     20380 non-null  object
 1   영문명     20380 non-null  object
dtypes: object(2)
memory usage: 477.7+ KB


In [8]:
ing_df.reset_index(inplace=True)
ing_df.head(10)

,성분코드,성분명,영문명
0,6,류코노스톡/무발효여과물,Leuconostoc/Radish Root Ferment Filtrate
1,7,백혈구추출물,Leukocyte Extract
2,8,리나칸투스 콤무니스추출물,Rhinacanthus Communis Extract
3,9,리날룰,Linalool
4,10,리날릴아세테이트,Linalyl Acetate
5,11,리놀레닉애씨드,Linolenic Acid
6,12,리놀레아마이드엠이에이,Linoleamide MEA
7,13,구상나무잎가루,Abies Koreana Leaf Powder
8,14,리놀레아미도프로필다이메틸아민다이머다이리놀리에이트,Linoleamidopropyl Dimethylamine Dimer Dilinoleate
9,15,리놀레아미도프로필에틸다이모늄에토설페이트,Linoleamidopropyl Ethyldimonium Ethosulfate


In [9]:
# inci-decoder에 성분을 검색할 때, format이 맞지 않는 표기명들이 있었다.
# 괄호와 그 안에 들어있는 내용을 제거하자
import re
pattern = r'\([^)]*\)'

for idx, row in ing_df.iterrows():
    tmp = ing_df.iloc[idx]['영문명']
    try:
        if '(' in tmp:
            txt = re.sub(pattern=pattern, repl='', string= tmp)
            txt = ' '.join(txt.split())
            ing_df.iloc[idx,2] = txt
    except:
        pass

In [10]:
ing_df['formatted_영문명'] = ing_df['영문명'].str.lower().str.replace(" ","-").str.replace("/","-") 
# inci-decoder에 검색가능한 format으로 변경하여 컬럼 추가

In [11]:
ing_df.head(10)

,성분코드,성분명,영문명,formatted_영문명
0,6,류코노스톡/무발효여과물,Leuconostoc/Radish Root Ferment Filtrate,leuconostoc-radish-root-ferment-filtrate
1,7,백혈구추출물,Leukocyte Extract,leukocyte-extract
2,8,리나칸투스 콤무니스추출물,Rhinacanthus Communis Extract,rhinacanthus-communis-extract
3,9,리날룰,Linalool,linalool
4,10,리날릴아세테이트,Linalyl Acetate,linalyl-acetate
5,11,리놀레닉애씨드,Linolenic Acid,linolenic-acid
6,12,리놀레아마이드엠이에이,Linoleamide MEA,linoleamide-mea
7,13,구상나무잎가루,Abies Koreana Leaf Powder,abies-koreana-leaf-powder
8,14,리놀레아미도프로필다이메틸아민다이머다이리놀리에이트,Linoleamidopropyl Dimethylamine Dimer Dilinoleate,linoleamidopropyl-dimethylamine-dimer-dilinoleate
9,15,리놀레아미도프로필에틸다이모늄에토설페이트,Linoleamidopropyl Ethyldimonium Ethosulfate,linoleamidopropyl-ethyldimonium-ethosulfate


In [14]:
ing_df.to_csv('../data/ing_name.csv')